#  Alpado Kyujanggak (알파코 규장각, 가제)
- 권장 실행 환경: Google Colab
- Data: `ffinal_data.pkl` (type: pandas.core.frame.DataFrame)
    - 교보문고 2023년 12월 기준 분야별 베스트셀러

## <환경설치>

In [ ]:
# 퍄키지 설치
!pip install konlpy # konlpy 설치
!pip install summa # 텍스트 요약
# !pip install -U --no-cache-dir gdown --pre
# !gdown --no-cookies --id 1KuBG40WNpVPV1ilfGaiCI2D3l8JVSfbS

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54388 sha256=7d8fa3a14ddccc51422658c0d921d3508e3540381afa01636b9f8d57eb580f10
  Stored in directory: /root/.cache/pip/wheels/4a/ca/c5/4958614cfba88ed6ceb7cb5a849f9f89f9ac49971616bc919f
Successfully built summa


In [ ]:
# 라이브러리 불러오기 및 객체 생성
from konlpy.tag import Hannanum, Okt, Kkma
from summa import keywords
from sklearn.metrics import jaccard_score
# import gensim
# from gensim.models.word2vec import Word2Vec
import re
import pandas as pd
from tqdm import tqdm, tqdm_pandas
import gensim
import pickle

tqdm_pandas(tqdm())
hannanum = Hannanum()
kkma = Kkma()
okt = Okt()

0it [00:00, ?it/s]


In [ ]:
with open('ffinal_data.pkl', 'rb') as f :
    li = pickle.load(f)

## 함수 정의

In [ ]:
stopwords = ['저자', '작가', '소개', '인기', '우리', '계기', '보유', '내용', '이야기'] # 불용어

In [ ]:
# 키워드 추출 함수
def preprocess(string) :
    string = ' '.join(re.findall('[가-힣]+|[a-z]+', string.lower())) # 특수문자들 제거 및 모든 영어 소문자 변환
    string = okt.nouns(string)
    for ind, i in enumerate(string) :
        if i in stopwords :
            string[ind] = ''
    string = ' '.join(string)
    string = ' '.join(re.findall('\w{2,}', string.lower())) # 1글자인 단어들 삭제
    words_n = 5 # 뽑을 키워드 갯수

    if len(set(string.split())) < words_n :
        words_n = len(set(string.split())) # 단어 갯수가 적을 경우 뽑을 키워드 갯수를 낮춘다

    string = keywords.keywords(string, words = words_n, scores = True) # 키워드 뽑기

    word = []
    score = []
    for i, j in string :
        i = ''.join(i).split()
        word.append(i)
    word_list = []
    for i in word :
        for j in i :
            word_list.append(j)
    for i in word_list : # 단어들 1개씩
        for ind, j in enumerate(word) :
            if i in j :
                score.append([i, round(string[ind][1], 2)])
    if len(score) > 5 :
        score = score[:5]

    return score

In [ ]:
# 키워드들을 딕셔너리 형태로 변환
def DicTrans(keyword) :

    intro = [i[0] for i in keyword]

    dic = {}
    for ind, i in enumerate(intro) :
        dic[i] = keyword[ind][1]

    return dic

In [ ]:
# 인풋데이터와, 책 리스트의 자카드 유사도 계산
def JarccardSimilarity (input_key, data_key) :
    score = 0

    in_keys = set(input_key.keys())
    da_keys = set(data_key.keys())
    same = in_keys & da_keys
    for i in list(same) :
        score += (input_key[i] * 1.7) + (data_key[i] * 1.3) # 가중치: 입력 1.7 , 데이터베이스 1.3
    a = set.union(in_keys ,da_keys)
    return score / len(a)

In [ ]:
# 제목의 자카드 유사도 계산
def JarccardSimilarity_title (input_key, data_key) :
    score = 0

    in_keys = set(input_key.keys())
    da_keys = set(data_key)
    same = in_keys & da_keys
    for i in list(same) :
        score += (input_key[i] * 1.7) + 1.3
    a = set.union(in_keys, da_keys)
    return score / len(a)

In [ ]:
# 사용자가 넣은 문장을 전처리 하여, 데이터 셋과 유사도 분석 실행, 상위 n개 출력
def Recommendation(input_text, data_set, Book_Count) :

    input_text = preprocess(input_text)
    input_text = DicTrans(input_text)

    intro_score = data_set['소개글_키워드'].apply(lambda a : JarccardSimilarity(input_text, a))
    index_score = data_set['목차_키워드'].apply(lambda a : JarccardSimilarity(input_text, a))
    title_score = data_set['상품명_Hannanum'].apply(lambda a : JarccardSimilarity_title(input_text, a))

    intro_sim_list = intro_score.sort_values()[-1 : -(Book_Count + 1) : -1] # Book_Count 갯수만큼 책을 뽑아준다
    index_sim_list = index_score.sort_values()[-1 : -(Book_Count + 1) : -1]
    title_sim_list = title_score.sort_values()[-1 : -(Book_Count + 1) : -1]

    for title , t in zip(['소개글 기반 유사도', '목차 기반 유사도', '제목 기반 유사도'], [intro_sim_list, index_sim_list, title_sim_list]) :
        print('==================================================================================================================')
        print(title)
        print('==================================================================================================================')
        print('==================================================================================================================')
        for key, score in zip(t.keys(), t) :
            if score == 0 :
                break
            for i in ['상품명','저자', '소개글', '목차', '분야', '정가'] :
                print(f'{i} : {data_set.loc[key][i]}')
            print(f'유사도 : {score}')
            print('==================================================================================================================')

## 실행파일

In [ ]:
# 사용자가 넣은 문장을 전처리 하여, 데이터 셋과 유사도 분석 실행, 상위 n개 출력
def Recommendation(input_text, data_set, Book_Count) :

    input_text = preprocess(input_text)
    input_text = DicTrans(input_text)

    intro_score = data_set['소개글_키워드'].apply(lambda a : JarccardSimilarity(input_text, a))
    index_score = data_set['목차_키워드'].apply(lambda a : JarccardSimilarity(input_text, a))
    title_score = data_set['상품명_Hannanum'].apply(lambda a : JarccardSimilarity_title(input_text, a))

    intro_sim_list = intro_score.sort_values()[-1 : -(Book_Count + 1) : -1] # Book_Count 갯수만큼 책을 뽑아준다
    index_sim_list = index_score.sort_values()[-1 : -(Book_Count + 1) : -1]
    title_sim_list = title_score.sort_values()[-1 : -(Book_Count + 1) : -1]

    for title , t in zip(['소개글 기반 유사도', '목차 기반 유사도', '제목 기반 유사도'], [intro_sim_list, index_sim_list, title_sim_list]) :
        print()
        print('<' + title + '>')
        print()
        print()

        for key, score in zip(t.keys(), t) :
            if score == 0 :
                break
            for i in ['상품명','저자', '소개글', '목차', '분야', '정가'] :
                print(f'{i} : {data_set.loc[key][i]}')
            print(f'유사도 : {score}')
            print()

In [ ]:
import pandas as pd

# 사용자가 넣은 문장을 전처리 하여, 데이터 셋과 유사도 분석 실행, 상위 n개 출력
def Recommendation(input_text, data_set, Book_Count):

    input_text = preprocess(input_text)
    input_text = DicTrans(input_text)

    intro_score = data_set['소개글_키워드'].apply(lambda a: JarccardSimilarity(input_text, a))
    index_score = data_set['목차_키워드'].apply(lambda a: JarccardSimilarity(input_text, a))
    title_score = data_set['상품명_Hannanum'].apply(lambda a: JarccardSimilarity_title(input_text, a))

    intro_sim_list = intro_score.sort_values()[-1: -(Book_Count + 1): -1]  # Book_Count 갯수만큼 책을 뽑아준다
    index_sim_list = index_score.sort_values()[-1: -(Book_Count + 1): -1]
    title_sim_list = title_score.sort_values()[-1: -(Book_Count + 1): -1]

    result_df = pd.DataFrame(columns=['상품명', '저자', '소개글', '목차', '분야', '정가', '유사도'])

    for title, t in zip(['소개글 기반 유사도', '목차 기반 유사도', '제목 기반 유사도'], [intro_sim_list, index_sim_list, title_sim_list]):
        print()
        print('<' + title + '>')
        print()

        for key, score in zip(t.keys(), t):
            if score == 0:
                break
            book_info = data_set.loc[key, ['상품명', '저자', '소개글', '목차', '분야', '정가']]
            # 결과를 데이터프레임으로 출력
            result_df = result_df.append(pd.Series(list(book_info) + [score], index=result_df.columns), ignore_index=True)


    print()
    print(result_df)


In [ ]:
import pandas as pd

# 사용자가 넣은 문장을 전처리 하여, 데이터 셋과 유사도 분석 실행, 상위 n개 출력
def Recommendation(input_text, data_set, Book_Count):

    input_text = preprocess(input_text)
    input_text = DicTrans(input_text)

    intro_score = data_set['소개글_키워드'].apply(lambda a: JarccardSimilarity(input_text, a))
    index_score = data_set['목차_키워드'].apply(lambda a: JarccardSimilarity(input_text, a))
    title_score = data_set['상품명_Hannanum'].apply(lambda a: JarccardSimilarity_title(input_text, a))

    intro_sim_list = intro_score.sort_values()[-1: -(Book_Count + 1): -1]  # Book_Count 갯수만큼 책을 뽑아준다
    index_sim_list = index_score.sort_values()[-1: -(Book_Count + 1): -1]
    title_sim_list = title_score.sort_values()[-1: -(Book_Count + 1): -1]

    result_df = pd.DataFrame(columns=['상품명', '저자', '소개글', '목차', '분야', '정가', '유사도'])

    for title, t in zip(['소개글 기반 유사도', '목차 기반 유사도', '제목 기반 유사도'], [intro_sim_list, index_sim_list, title_sim_list]):
        print()
        print('<' + title + '>')
        print()

        for key, score in zip(t.keys(), t):
            if score == 0:
                break
            book_info = data_set.loc[key, ['상품명', '저자', '소개글', '목차', '분야', '정가']]
            result_df = result_df.append(pd.Series(list(book_info) + [score], index=result_df.columns), ignore_index=True)

    # 결과를 데이터프레임으로 출력
    print()
    print(result_df)

In [ ]:
# 사용자가 넣은 문장을 전처리 하여, 데이터 셋과 유사도 분석 실행, 상위 n개 출력
def Recommendation(input_text, data_set, Book_Count) :

    input_text = preprocess(input_text)
    input_text = DicTrans(input_text)

    intro_score = data_set['소개글_키워드'].apply(lambda a : JarccardSimilarity(input_text, a))  # 시리즈
    index_score = data_set['목차_키워드'].apply(lambda a : JarccardSimilarity(input_text, a))
    title_score = data_set['상품명_Hannanum'].apply(lambda a : JarccardSimilarity_title(input_text, a))

    intro_sim_list = intro_score.sort_values()[-1 : -(Book_Count + 1) : -1] # Book_Count 갯수만큼 책을 뽑아준다
    index_sim_list = index_score.sort_values()[-1 : -(Book_Count + 1) : -1]
    title_sim_list = title_score.sort_values()[-1 : -(Book_Count + 1) : -1]

    for title , t in zip(['소개글 기반 유사도', '목차 기반 유사도', '제목 기반 유사도'], [intro_sim_list, index_sim_list, title_sim_list]) :
        print('==================================================================================================================')
        print(title)
        print('==================================================================================================================')
        print('==================================================================================================================')
        for key, score in zip(t.keys(), t) :
            if score == 0 :
                break
            for i in ['상품명','저자', '소개글', '목차', '분야', '정가'] :
                print(f'{i} : {data_set.loc[key][i]}')
            print(f'유사도 : {score}')
            print('==================================================================================================================')

In [ ]:
# 04시 01분
import pandas as pd

def Recommendation(input_text, data_set, Book_Count):

    input_text = preprocess(input_text)
    input_text = DicTrans(input_text)

    intro_score = data_set['소개글_키워드'].apply(lambda a: JarccardSimilarity(input_text, a))
    index_score = data_set['목차_키워드'].apply(lambda a: JarccardSimilarity(input_text, a))
    title_score = data_set['상품명_Hannanum'].apply(lambda a: JarccardSimilarity_title(input_text, a))

    intro_sim_list = intro_score.sort_values()[-1: -(Book_Count + 1): -1]  # Book_Count 갯수만큼 책을 뽑아준다
    index_sim_list = index_score.sort_values()[-1: -(Book_Count + 1): -1]
    title_sim_list = title_score.sort_values()[-1: -(Book_Count + 1): -1]

    result_df = pd.DataFrame()

    for title, t in zip(['소개글 기반 유사도', '목차 기반 유사도', '제목 기반 유사도'], [intro_sim_list, index_sim_list, title_sim_list]):
        for key, score in zip(t.keys(), t):
            if score == 0:
                continue
            book_info = data_set.loc[key, ['상품명', '저자', '소개글', '목차', '분야', '정가']]
            book_info['유사도'] = score
            book_info['유사도 기준'] = title
            result_df = result_df.append(book_info, ignore_index=True)

    # 모든 결과를 하나의 데이터프레임으로 병합
    return display(result_df)

# 함수 사용 예시
# result = Recommendation('사용자 입력 문장', 데이터셋, 추천할 책의 개수)
# print(result)


In [ ]:
def Recommendation(input_text, data_set, Book_Count):

    input_text = preprocess(input_text)
    input_text = DicTrans(input_text)

    intro_score = data_set['소개글_키워드'].apply(lambda a: JarccardSimilarity(input_text, a))
    index_score = data_set['목차_키워드'].apply(lambda a: JarccardSimilarity(input_text, a))
    title_score = data_set['상품명_Hannanum'].apply(lambda a: JarccardSimilarity_title(input_text, a))

    intro_sim_list = intro_score.sort_values()[-1: -(Book_Count + 1): -1]  # Book_Count 갯수만큼 책을 뽑아준다
    index_sim_list = index_score.sort_values()[-1: -(Book_Count + 1): -1]
    title_sim_list = title_score.sort_values()[-1: -(Book_Count + 1): -1]


    result_df = pd.DataFrame()

    for title, t in zip(['소개글 기반 유사도', '목차 기반 유사도', '제목 기반 유사도'], [intro_sim_list, index_sim_list, title_sim_list]):
        for key, score in zip(t.keys(), t):
            if score == 0:
                continue
            book_info = data_set.loc[key, ['상품명', '저자', '소개글', '목차', '분야', '정가']]
            book_info['유사도'] = score
            book_info['유사도 기준'] = title

            # result_df에 행 추가
            result_df = pd.concat([result_df, pd.DataFrame([book_info])], ignore_index=True)

    # 모든 결과를 하나의 데이터프레임으로 병합
    return result_df

# 함수 사용 예시
# result = Recommendation('사용자 입력 문장', 데이터셋, 추천할 책의 개수)
# print(result)


In [ ]:
def Recommendation(input_text, data_set, Book_Count):

    input_text = preprocess(input_text)
    input_text = DicTrans(input_text)

    intro_score = data_set['소개글_키워드'].apply(lambda a: JarccardSimilarity(input_text, a))
    index_score = data_set['목차_키워드'].apply(lambda a: JarccardSimilarity(input_text, a))
    title_score = data_set['상품명_Hannanum'].apply(lambda a: JarccardSimilarity_title(input_text, a))

    intro_sim_list = intro_score.sort_values()[-1: -(Book_Count + 1): -1]  # Book_Count 갯수만큼 책을 뽑아준다
    index_sim_list = index_score.sort_values()[-1: -(Book_Count + 1): -1]
    title_sim_list = title_score.sort_values()[-1: -(Book_Count + 1): -1]

    result_df = pd.DataFrame()

    for title, t in zip(['소개글 기반 유사도', '목차 기반 유사도', '제목 기반 유사도'], [intro_sim_list, index_sim_list, title_sim_list]):
        for key, score in zip(t.keys(), t):
            if score == 0:
                continue
            book_info = data_set.loc[key, ['상품명', '저자', '소개글', '목차', '분야', '정가']]
            book_info['유사도'] = score
            book_info['유사도 기준'] = title

            # result_df에 행 추가
            result_df = pd.concat([result_df, pd.DataFrame([book_info])], ignore_index=True)

    # 모든 결과를 하나의 데이터프레임으로 병합
    return result_df

# 함수 사용 예시
# result = Recommendation('사용자 입력 문장', 데이터셋, 추천할 책의 개수)
# print(result)


In [ ]:
import pandas as pd

def Recommendation(input_text, data_set, Book_Count):

    input_text = preprocess(input_text)
    input_text = DicTrans(input_text)

    intro_score = data_set['소개글_키워드'].apply(lambda a: JarccardSimilarity(input_text, a))
    index_score = data_set['목차_키워드'].apply(lambda a: JarccardSimilarity(input_text, a))
    title_score = data_set['상품명_Hannanum'].apply(lambda a: JarccardSimilarity_title(input_text, a))

    intro_sim_list = intro_score.sort_values()[-1: -(Book_Count + 1): -1]  # Book_Count 갯수만큼 책을 뽑아준다
    index_sim_list = index_score.sort_values()[-1: -(Book_Count + 1): -1]
    title_sim_list = title_score.sort_values()[-1: -(Book_Count + 1): -1]

    # 각 유사도 기준에 따른 결과를 저장할 데이터프레임 초기화
    intro_df = pd.DataFrame()
    index_df = pd.DataFrame()
    title_df = pd.DataFrame()

    for title, t, result_df in zip(['소개글 기반 유사도', '목차 기반 유사도', '제목 기반 유사도'],
                                   [intro_sim_list, index_sim_list, title_sim_list],
                                   [intro_df, index_df, title_df]):
        for key, score in zip(t.keys(), t):
            if score == 0:
                continue
            book_info = data_set.loc[key, ['상품명', '저자', '소개글', '목차', '분야', '정가']]
            book_info['유사도'] = score
            book_info['유사도 기준'] = title

            # 각 유사도 기준에 따른 데이터프레임에 행 추가
            result_df = pd.concat([result_df, pd.DataFrame([book_info])], ignore_index=True)

    # 각 유사도 기준에 따른 결과 데이터프레임 반환
    return intro_df, index_df, title_df

# 함수 사용 예시
# intro_df, index_df, title_df = Recommendation('사용자 입력 문장', 데이터셋, 추천할 책의 개수)
# print('소개글 기반 유사도 결과:')
# print(intro_df)
# print('목차 기반 유사도 결과:')
# print(index_df)
# print('제목 기반 유사도 결과:')
# print(title_df)


In [ ]:
text = """이 과제는 인류학적 접근을 통해 자본주의 경제 시스템이 인간 사회와 문화에 미치는 다양하고 복잡한
 영향을 체계적으로 탐구하며, 학생들은 자본주의의 형성과 확산, 소비문화, 불평등, 글로벌화, 환경파괴 등 다양한
  측면에서 인류학의 독특한 관점을 활용하여 현대 사회의 복잡한 사회 경제적 문제와 그 영향을 사회학적,
  문화인류학적, 공동체적 시각에서 심층적으로 이해하고 분석하며, 인간의 역사와 문화와 함께 성장한 자본주의와의
  관계에 대한 통찰력을 개발하고자 하는 목적을 달성하기 위한 레포트를 작성하도록 합니다."""

In [ ]:
# 책 리스트 불러오기
with open('ffinal_data.pkl', 'rb') as f:
    Books = pickle.load(f).reset_index(drop=True)

# 입력 받기
Book_Field = int(input("분야를 입력 하시겠습니까? (True(입력): 1, False(미입력): 0): "))
print()

if bool(Book_Field):
    print('<분야 입력 제한 범위> \n', Books['분야'].unique())
    print()
    Your_Field = input("분야를 입력 해주세요: ")
    book_list = Books[Books['분야'] == Your_Field]
    print()
else:
    book_list = Books

Book_Count = int(input("추천 받을 책 권수를 입력해주세요 (0 이상의 정수): "))
print()

Your_Text = input('과제를 입력 해주세요: ')
print()

your_keyword = preprocess(Your_Text) # 입력 문장에서 키워드 추출

# 키워드 출력 구문
print('<입렵 문장의 키워드>')
for i in range(len(your_keyword)):
    print(your_keyword[i][0], your_keyword[i][1], sep=' : ')
print()

# 유사도 추출 후, 책 추천
if Your_Field :

    field_data = Book[Book['분야'] == Your_Field]
    not_field_data = Book[Book['분야'] != Your_Field]

    print(f'선택하신 분야: {Your_Field}의 도서 추천 리스트 입니다')
    result_fd = Recommendation(Your_Text, field_data, Book_Count)
    display(result_fd)
    print()

    print('선택하신 분야 외의 도서 추천 리스트 입니다')
    result_re_fd = Recommendation(Your_Text, not_field_data, Book_Count)
    display(result_re_fd)

else :
    print('도서 추천 리스트 입니다')
    result_all =  Recommendation(Your_Text, Book, Book_Count)
    display(result_all)

분야를 입력 하시겠습니까? (True(입력): 1, False(미입력): 0): 0

추천 받을 책 권수를 입력해주세요 (0 이상의 정수): 6

과제를 입력 해주세요: 이 과제는 인류학적 접근을 통해 자본주의 경제 시스템이 인간 사회와 문화에 미치는 다양하고 복잡한 영향을 체계적으로 탐구하며, 학생들은 자본주의의 형성과 확산, 소비문화, 불평등, 글로벌화, 환경파괴 등 다양한 측면에서 인류학의 독특한 관점을 활용하여 현대 사회의 복잡한 사회 경제적 문제와 그 영향을 사회학적, 문화인류학적, 공동체적 시각에서 심층적으로 이해하고 분석하며, 인간의 역사와 문화와 함께 성장한 자본주의와의 관계에 대한 통찰력을 개발하고자 하는 목적을 달성하기 위한 레포트를 작성하도록 합니다.

<입렵 문장의 키워드>
인류학 : 0.33
자본주의 : 0.3
문화 : 0.29
영향 : 0.29
인간 : 0.19



NameError: ignored

In [ ]:
your_keyword

[['인류학', 0.33], ['자본주의', 0.3], ['문화', 0.29], ['영향', 0.29], ['인간', 0.19]]

In [ ]:
import pandas as pd

def Recommendation(input_text, data_set, Book_Count):

    input_text = preprocess(input_text)
    input_text = DicTrans(input_text)

    intro_score = data_set['소개글_키워드'].apply(lambda a: JarccardSimilarity(input_text, a))
    index_score = data_set['목차_키워드'].apply(lambda a: JarccardSimilarity(input_text, a))
    title_score = data_set['상품명_Hannanum'].apply(lambda a: JarccardSimilarity_title(input_text, a))

    intro_sim_list = intro_score.sort_values()[-1: -(Book_Count + 1): -1]  # Book_Count 갯수만큼 책을 뽑아준다
    index_sim_list = index_score.sort_values()[-1: -(Book_Count + 1): -1]
    title_sim_list = title_score.sort_values()[-1: -(Book_Count + 1): -1]

    # 각 유사도 기준에 따른 결과를 저장할 데이터프레임 초기화
    intro_df = pd.DataFrame()
    index_df = pd.DataFrame()
    title_df = pd.DataFrame()

    for title, t, result_df in zip(['소개글 기반 유사도', '목차 기반 유사도', '제목 기반 유사도'],
                                   [intro_sim_list, index_sim_list, title_sim_list],
                                   [intro_df, index_df, title_df]):
        for key, score in zip(t.keys(), t):
            if score == 0:
                continue
            book_info = data_set.loc[key, ['상품명', '저자', '소개글', '목차', '분야', '정가']]
            book_info['유사도'] = score
            book_info['유사도 기준'] = title

            # 각 유사도 기준에 따른 데이터프레임에 행 추가
            result_df = pd.concat([result_df, pd.DataFrame([book_info])], ignore_index=True)

    # 각 유사도 기준에 따른 결과 데이터프레임 반환
    return intro_df, index_df, title_df

# 함수 사용 예시
# intro_df, index_df, title_df = Recommendation('사용자 입력 문장', 데이터셋, 추천할 책의 개수)
# print('소개글 기반 유사도 결과:')
# print(intro_df)
# print('목차 기반 유사도 결과:')
# print(index_df)
# print('제목 기반 유사도 결과:')
# print(title_df)


In [ ]:
# 책 리스트 불러오기
with open('ffinal_data.pkl', 'rb') as f:
    Books = pickle.load(f).reset_index(drop=True)

# 입력 받기
Book_Field = int(input("분야를 입력 하시겠습니까? (True(입력): 1, False(미입력): 0): "))
print()

if bool(Book_Field):
    print('<분야 입력 제한 범위> \n', Books['분야'].unique())
    print()
    Your_Field = input("분야를 입력 해주세요: ")
    book_list = Books[Books['분야'] == Your_Field]
    print()
else:
    book_list = Books

Book_Count = int(input("추천 받을 책 권수를 입력해주세요 (0 이상의 정수): "))
print()

Your_Text = input('과제를 입력 해주세요: ')
print()

your_keyword = preprocess(Your_Text) # 입력 문장에서 키워드 추출

# 키워드 출력 구문
print('<입렵 문장의 키워드>')
for i in range(len(your_keyword)):
    print(your_keyword[i][0], your_keyword[i][1], sep=' : ')
print()

# 유사도 추출 후, 책 추천
if Your_Field :

    field_data = Book[Book['분야'] == Your_Field]
    not_field_data = Book[Book['분야'] != Your_Field]

    print(f'선택하신 분야: {Your_Field}의 도서 추천 리스트 입니다')
    result_fd = Recommendation(Your_Text, field_data, Book_Count)
    display(result_fd)
    print()

    print('선택하신 분야 외의 도서 추천 리스트 입니다')
    result_re_fd = Recommendation(Your_Text, not_field_data, Book_Count)
    display(result_re_fd)

else :
    print('도서 추천 리스트 입니다')
    result_all =  Recommendation(Your_Text, Book, Book_Count)
    display(result_all)

In [ ]:
text = """이 과제는 인류학적 접근을 통해 자본주의 경제 시스템이 인간 사회와 문화에 미치는 다양하고 복잡한
 영향을 체계적으로 탐구하며, 학생들은 자본주의의 형성과 확산, 소비문화, 불평등, 글로벌화, 환경파괴 등 다양한
  측면에서 인류학의 독특한 관점을 활용하여 현대 사회의 복잡한 사회 경제적 문제와 그 영향을 사회학적,
  문화인류학적, 공동체적 시각에서 심층적으로 이해하고 분석하며, 인간의 역사와 문화와 함께 성장한 자본주의와의
  관계에 대한 통찰력을 개발하고자 하는 목적을 달성하기 위한 레포트를 작성하도록 합니다."""

In [ ]:
def Recommendation(input_text, data_set, Book_Count):

    input_text = preprocess(input_text)
    input_text = DicTrans(input_text)

    intro_score = data_set['소개글_키워드'].apply(lambda a: JarccardSimilarity(input_text, a))
    index_score = data_set['목차_키워드'].apply(lambda a: JarccardSimilarity(input_text, a))
    title_score = data_set['상품명_Hannanum'].apply(lambda a: JarccardSimilarity_title(input_text, a))

    intro_sim_list = intro_score.sort_values()[-1: -(Book_Count + 1): -1]  # Book_Count 갯수만큼 책을 뽑아준다
    index_sim_list = index_score.sort_values()[-1: -(Book_Count + 1): -1]
    title_sim_list = title_score.sort_values()[-1: -(Book_Count + 1): -1]

    # 각 유사도 기준에 따른 데이터프레임 생성
    intro_df = pd.DataFrame()
    index_df = pd.DataFrame()
    title_df = pd.DataFrame()

    for title, t, result_df in zip(['소개글 기반 유사도', '목차 기반 유사도', '제목 기반 유사도'],
                                   [intro_sim_list, index_sim_list, title_sim_list],
                                   [intro_df, index_df, title_df]):
        for key, score in zip(t.keys(), t):
            if score == 0:
                continue
            book_info = data_set.loc[key, ['상품명', '저자', '소개글', '목차', '분야', '정가']]
            book_info['유사도'] = score
            book_info['유사도 기준'] = title

            result_df = pd.concat([result_df, pd.DataFrame([book_info])], ignore_index=True)

    return intro_df, index_df, title_df

In [ ]:
import pickle

# 책 리스트 불러오기
with open('ffinal_data.pkl', 'rb') as f:
    Books = pickle.load(f).reset_index(drop=True)

# 입력 받기
Book_Field = int(input("분야를 입력 하시겠습니까? (True(입력): 1, False(미입력): 0): "))
print()

if bool(Book_Field):
    print('<분야 입력 제한 범위> \n', Books['분야'].unique())
    print()
    Your_Field = input("분야를 입력 해주세요: ")
    book_list = Books[Books['분야'] == Your_Field]
    print()
else:
    book_list = Books

Book_Count = int(input("추천 받을 책 권수를 입력해주세요 (0 이상의 정수): "))
print()

Your_Text = input('과제를 입력 해주세요: ')
print()

your_keyword = preprocess(Your_Text) # 입력 문장에서 키워드 추출

# 키워드 출력 구문
print('<입력 문장의 키워드>')
for i in range(len(your_keyword)):
    print(your_keyword[i][0], your_keyword[i][1], sep=' : ')
print()

# 유사도 추출 후, 책 추천
if Your_Field:
    field_data = Books[Books['분야'] == Your_Field]
    not_field_data = Books[Books['분야'] != Your_Field]

    print(f'선택하신 분야: {Your_Field}의 도서 추천 리스트 입니다')
    intro_fd, index_fd, title_fd = Recommendation(Your_Text, field_data, Book_Count)

    print('소개글 유사도 기반')
    display(intro_fd)
    print()
    print('목차 유사도 기반')
    display(index_fd)
    print()
    print('제목 유사도 기반')
    display(title_fd)
    print()

    print('선택하신 분야 외의 도서 추천 리스트 입니다')
    intro_re_fd, index_re_fd, title_re_fd = Recommendation(Your_Text, not_field_data, Book_Count)

    print('소개글 유사도 기반')
    display(intro_fd)
    print()
    print('목차 유사도 기반')
    display(index_fd)
    print()
    print('제목 유사도 기반')
    display(title_fd)
    print()

else:
    print('도서 추천 리스트 입니다')
    intro_all, index_all, title_all = Recommendation(Your_Text, Books, Book_Count)

    print('소개글 유사도 기반')
    display(intro_fd)
    print()
    print('목차 유사도 기반')
    display(index_fd)
    print()
    print('제목 유사도 기반')
    display(title_fd)
    print()


In [ ]:
# 키워드 추출 함수
def preprocess(string) :
    string = ' '.join(re.findall('[가-힣]+|[a-z]+', string.lower())) # 특수문자들 제거 및 모든 영어 소문자 변환
    string = okt.nouns(string)
    for ind, i in enumerate(string) :
        if i in stopwords :
            string[ind] = ''
    string = ' '.join(string)
    string = ' '.join(re.findall('\w{2,}', string.lower())) # 1글자인 단어들 삭제
    words_n = 5 # 뽑을 키워드 갯수

    if len(set(string.split())) < words_n :
        words_n = len(set(string.split())) # 단어 갯수가 적을 경우 뽑을 키워드 갯수를 낮춘다

    string = keywords.keywords(string, words = words_n, scores = True) # 키워드 뽑기

    word = []
    score = []
    for i, j in string :
        i = ''.join(i).split()
        word.append(i)
    word_list = []
    for i in word :
        for j in i :
            word_list.append(j)
    for i in word_list : # 단어들 1개씩
        for ind, j in enumerate(word) :
            if i in j :
                score.append([i, round(string[ind][1], 2)])
    if len(score) > 5 :
        score = score[:5]

    return score

# 키워드들을 딕셔너리 형태로 변환
def DicTrans(keyword) :

    intro = [i[0] for i in keyword]

    dic = {}
    for ind, i in enumerate(intro) :
        dic[i] = keyword[ind][1]

    return dic

# 인풋데이터와, 책 리스트의 자카드 유사도 계산
def JarccardSimilarity (input_key, data_key) :
    score = 0

    in_keys = set(input_key.keys())
    da_keys = set(data_key.keys())
    same = in_keys & da_keys
    for i in list(same) :
        score += (input_key[i] * 1.7) + (data_key[i] * 1.3) # 가중치: 입력 1.7 , 데이터베이스 1.3
    a = set.union(in_keys ,da_keys)
    return score / len(a)

# 제목의 자카드 유사도 계산
def JarccardSimilarity_title (input_key, data_key) :
    score = 0

    in_keys = set(input_key.keys())
    da_keys = set(data_key)
    same = in_keys & da_keys
    for i in list(same) :
        score += (input_key[i] * 1.7) + 1.3
    a = set.union(in_keys, da_keys)
    return score / len(a)

In [ ]:
# 키워드 추출 함수
def preprocess(string) :
    string = ' '.join(re.findall('[가-힣]+|[a-z]+', string.lower())) # 특수문자들 제거 및 모든 영어 소문자 변환
    string = okt.nouns(string)
    for ind, i in enumerate(string) :
        if i in stopwords :
            string[ind] = ''
    string = ' '.join(string)
    string = ' '.join(re.findall('\w{2,}', string)) # 1글자인 단어들 삭제
    words_n = 5 # 뽑을 키워드 갯수

    if len(set(string.split())) < words_n :
        words_n = len(set(string.split())) # 단어 갯수가 적을 경우 뽑을 키워드 갯수를 낮춘다

    string = keywords.keywords(string, words = words_n, scores = True) # 키워드 뽑기

    word = []
    score = []
    for i, j in string :
        i = ''.join(i).split()
        word.append(i)
    word_list = []
    for i in word :
        for j in i :
            word_list.append(j)
    for i in word_list : # 단어들 1개씩
        for ind, j in enumerate(word) :
            if i in j :
                score.append([i, round(string[ind][1], 2)])
    if len(score) > 5 :
        score = score[:5]

    return score

# 키워드들을 딕셔너리 형태로 변환
def DicTrans(keyword) :

    intro = [i[0] for i in keyword]

    dic = {}
    for ind, i in enumerate(intro) :
        dic[i] = keyword[ind][1]

    return dic


# 인풋데이터와, 책 리스트의 자카드 유사도 계산
def JarccardSimilarity (input_key, data_key) :
    score = 0

    in_keys = set(input_key.keys())
    da_keys = set(data_key.keys())
    same = in_keys & da_keys
    for i in list(same) :
        score += (input_key[i] * 1.7) + (data_key[i] * 1.3) # 가중치: 입력 1.7 , 데이터베이스 1.3
    a = set.union(in_keys ,da_keys)
    return score / len(a)

# 제목의 자카드 유사도 계산
def JarccardSimilarity_title (input_key, data_key) :
    score = 0

    in_keys = set(input_key.keys())
    da_keys = set(data_key)
    same = in_keys & da_keys
    for i in list(same) :
        score += (input_key[i] * 1.7) + 1.3
    a = set.union(in_keys, da_keys)
    return score / len(a)




def Recommendation(input_text, data_set, Book_Count):

    input_text = preprocess(input_text)

    intro_score = data_set['소개글_키워드'].apply(lambda a: JarccardSimilarity(input_text, a))
    index_score = data_set['목차_키워드'].apply(lambda a: JarccardSimilarity(input_text, a))
    title_score = data_set['상품명_Hannanum'].apply(lambda a: JarccardSimilarity_title(input_text, a))

    intro_sim_list = intro_score.sort_values(ascending=False)[:Book_Count]
    index_sim_list = index_score.sort_values(ascending=False)[:Book_Count]
    title_sim_list = title_score.sort_values(ascending=False)[:Book_Count]

    # 각 유사도 기준에 따른 데이터프레임 생성
    intro_df = pd.DataFrame()
    index_df = pd.DataFrame()
    title_df = pd.DataFrame()

    for title, t, result_df in zip(['소개글 기반 유사도', '목차 기반 유사도', '제목 기반 유사도'],
                                   [intro_sim_list, index_sim_list, title_sim_list],
                                   [intro_df, index_df, title_df]):
        for key, score in zip(t.index, t):
            if score == 0:
                continue
            book_info = data_set.loc[key, ['상품명', '저자', '소개글', '목차', '분야', '정가']]
            book_info['유사도'] = score
            book_info['유사도 기준'] = title

            result_df = pd.concat([result_df, pd.DataFrame([book_info])], ignore_index=True)

    return intro_df, index_df, title_df

In [ ]:
# 인풋데이터와, 책 리스트의 자카드 유사도 계산
def JarccardSimilarity (input_key, data_key) :
    score = 0

    in_keys = set(input_key.keys())
    da_keys = set(data_key.keys())
    same = in_keys & da_keys
    for i in list(same) :
        score += (input_key[i] * 1.7) + (data_key[i] * 1.3) # 가중치: 입력 1.7 , 데이터베이스 1.3
    a = set.union(in_keys ,da_keys)
    return score / len(a)

# 제목의 자카드 유사도 계산
def JarccardSimilarity_title (input_key, data_key) :
    score = 0

    in_keys = set(input_key.keys())
    da_keys = set(data_key)
    same = in_keys & da_keys
    for i in list(same) :
        score += (input_key[i] * 1.7) + 1.3
    a = set.union(in_keys, da_keys)
    return score / len(a)

# 책 리스트 불러오기
with open('ffinal_data.pkl', 'rb') as f:
    Books = pickle.load(f).reset_index(drop=True)

# 입력 받기
Book_Field = int(input("분야를 입력 하시겠습니까? (True(입력): 1, False(미입력): 0): "))
print()

if bool(Book_Field):
    print('<분야 입력 제한 범위> \n', Books['분야'].unique())
    print()
    Your_Field = input("분야를 입력 해주세요: ")
    book_list = Books[Books['분야'] == Your_Field]
    print()
else:
    book_list = Books

Book_Count = int(input("추천 받을 책 권수를 입력해주세요 (0 이상의 정수): "))
print()

Your_Text = input('과제를 입력 해주세요: ')
print()

your_keyword = preprocess(Your_Text) # 입력 문장에서 키워드 추출

# 키워드 출력 구문
print('<입력 문장의 키워드>')
for i in range(len(your_keyword)):
    print(your_keyword[i][0], your_keyword[i][1], sep=' : ')
print()

# 유사도 추출 후, 책 추천
if bool(Book_Field):
    field_data = Books[Books['분야'] == Your_Field]
    not_field_data = Books[Books['분야'] != Your_Field]

    print(f'선택하신 분야: {Your_Field}의 도서 추천 리스트 입니다')

    intro_fd, index_fd, title_fd = Recommendation(Your_Text, field_data, Book_Count)

    print('소개글 유사도 기반')
    display(intro_fd)
    print()
    print('목차 유사도 기반')
    display(index_fd)
    print()
    print('제목 유사도 기반')
    display(title_fd)
    print()

    print('선택하신 분야 외의 도서 추천 리스트 입니다')
    intro_re_fd, index_re_fd, title_re_fd = Recommendation(Your_Text, not_field_data, Book_Count)

    print('소개글 유사도 기반')
    display(intro_re_fd)
    print()
    print('목차 유사도 기반')
    display(index_re_fd)
    print()
    print('제목 유사도 기반')
    display(title_re_fd)
    print()

else:
    print('도서 추천 리스트 입니다')
    intro_all, index_all, title_all = Recommendation(Your_Text, Books, Book_Count)
    print('소개글 유사도 기반')
    display(intro_all)
    print()
    print('목차 유사도 기반')
    display(index_all)
    print()
    print('제목 유사도 기반')
    display(title_all)
    print()

In [ ]:
def Recommendation(input_text, data_set, Book_Count):

    input_text = preprocess(input_text)
    input_text = DicTrans(input_text)

    intro_score = data_set['소개글_키워드'].apply(lambda a: JarccardSimilarity(input_text, a))
    index_score = data_set['목차_키워드'].apply(lambda a: JarccardSimilarity(input_text, a))
    title_score = data_set['상품명_Hannanum'].apply(lambda a: JarccardSimilarity_title(input_text, a))

    intro_sim_list = intro_score.sort_values(ascending=False)[:Book_Count]
    index_sim_list = index_score.sort_values(ascending=False)[:Book_Count]
    title_sim_list = title_score.sort_values(ascending=False)[:Book_Count]

    # 각 유사도 기준에 따른 데이터프레임 생성
    intro_df = []
    index_df = []
    title_df = []

    for title, t, result_df in zip(['소개글 기반 유사도', '목차 기반 유사도', '제목 기반 유사도'],
                                   [intro_sim_list, index_sim_list, title_sim_list],
                                   [intro_df, index_df, title_df]):
        for key, score in zip(t.index, t): # 인덱스 번호랑 유사도 점수
            if score == 0:
                continue
            book_info = []
            for i in ['상품명', '저자', '분야', '정가', '소개글', '목차'] :
                book_info.append(data_set.loc[key][i])
#            book_info = data_set.loc[key, '상품명', '저자', '소개글', '목차', '분야', '정가']
            book_info.append(float(score))
            book_info.append(title)
            result_df.append(book_info)
    intro_df = pd.DataFrame(intro_df, columns = ['상품명', '저자', '분야', '정가', '유사도', '유사도 기준', '소개글', '목차'])
    index_df = pd.DataFrame(index_df, columns = ['상품명', '저자', '분야', '정가', '유사도', '유사도 기준', '소개글', '목차'])
    title_df = pd.DataFrame(title_df, columns = ['상품명', '저자', '분야', '정가', '유사도', '유사도 기준', '소개글', '목차'])
    return intro_df, index_df, title_df

In [ ]:
# 책 리스트 불러오기
with open('ffinal_data.pkl', 'rb') as f:
    Books = pickle.load(f).reset_index(drop=True)

# 입력 받기
# Book_Field = int(input("분야를 입력 하시겠습니까? (True(입력): 1, False(미입력): 0): "))
Book_Field = 1
print()

if bool(Book_Field):
    print('<분야 입력 제한 범위> \n', Books['분야'].unique())
    print()
    # Your_Field = input("분야를 입력 해주세요: ")
    Your_Field = '인문'
    book_list = Books[Books['분야'] == Your_Field]
    print()
else:
    book_list = Books

# Book_Count = int(input("추천 받을 책 권수를 입력해주세요 (0 이상의 정수): "))
Book_Count =3
print()

# Your_Text = input('과제를 입력 해주세요: ')
text = """이 과제는 인류학적 접근을 통해 자본주의 경제 시스템이 인간 사회와 문화에 미치는 다양하고 복잡한
 영향을 체계적으로 탐구하며, 학생들은 자본주의의 형성과 확산, 소비문화, 불평등, 글로벌화, 환경파괴 등 다양한
  측면에서 인류학의 독특한 관점을 활용하여 현대 사회의 복잡한 사회 경제적 문제와 그 영향을 사회학적,
  문화인류학적, 공동체적 시각에서 심층적으로 이해하고 분석하며, 인간의 역사와 문화와 함께 성장한 자본주의와의
  관계에 대한 통찰력을 개발하고자 하는 목적을 달성하기 위한 레포트를 작성하도록 합니다."""
Your_Text = text
print()

your_keyword = preprocess(Your_Text) # 입력 문장에서 키워드 추출

# 키워드 출력 구문
print('<입력 문장의 키워드>')
for i in range(len(your_keyword)):
    print(your_keyword[i][0], your_keyword[i][1], sep=' : ')
print()

# 유사도 추출 후, 책 추천
if bool(Book_Field):
    field_data = Books[Books['분야'] == Your_Field]
    not_field_data = Books[Books['분야'] != Your_Field]

    print(f'선택하신 분야: {Your_Field}의 도서 추천 리스트 입니다')

    intro_fd, index_fd, title_fd = Recommendation(Your_Text, field_data, Book_Count)

    print('<소개글 유사도 기반>')
    print()
    display(intro_fd)
    print()
    print()
    print('<목차 유사도 기반>')
    print()
    display(index_fd)
    print()
    print()
    print('<제목 유사도 기반>')
    print()
    display(title_fd)
    print()

    print('선택하신 분야 외의 도서 추천 리스트 입니다')
    intro_re_fd, index_re_fd, title_re_fd = Recommendation(Your_Text, not_field_data, Book_Count)

    print('소개글 유사도 기반')
    display(intro_re_fd)
    print()
    print('목차 유사도 기반')
    display(index_re_fd)
    print()
    print('제목 유사도 기반')
    display(title_re_fd)
    print()

else:
    print('도서 추천 리스트 입니다')
    intro_all, index_all, title_all = Recommendation(Your_Text, Books, Book_Count)
    print('소개글 유사도 기반')
    display(intro_all)
    print()
    print('목차 유사도 기반')
    display(index_all)
    print()
    print('제목 유사도 기반')
    display(title_all)
    print()


<분야 입력 제한 범위> 
 ['예술/대중문화' '건강' '경제/경영' '역사/문화' '정치/사회' '청소년' '인문' '한국소개도서' '시/에세이'
 '취미/실용/스포츠' '어린이(초등)' '자기계발' '여행' '요리' '소설' '과학' '종교' '컴퓨터/IT' '기술/공학']




<입력 문장의 키워드>
인류학 : 0.33
자본주의 : 0.3
문화 : 0.29
영향 : 0.29
인간 : 0.19

선택하신 분야: 인문의 도서 추천 리스트 입니다
<소개글 유사도 기반>



,상품명,저자,소개글,목차,분야,정가,유사도,유사도 기준
0,감정의 재탄생,얀 플럼퍼,"감정이란 무엇인가? 학습되는가 아니면 인간 본질인가?\n고대부터 현재까지, ‘역사 ...",머리말\n\n들어가며 역사 그리고 감정\n01 감정의 메타역사\n02 인류학 : 사...,인문,28000,0,소개글 기반 유사도




<목차 유사도 기반>



,상품명,저자,소개글,목차,분야,정가,유사도,유사도 기준
0,로봇과 AI의 인류학,캐슬린 리처드슨,"로봇과 AI 그리고 인간,\n인류의 문화와 사회에 대해 근원적이고 본질적인 질문을 ...","감사의 말ㆍ7\n\n머리말: 절멸불안과 기계ㆍ11\n인류여, 너희의 종말을 경계하라...",인문,24000,0,목차 기반 유사도




<제목 유사도 기반>



...

,상품명,저자,소개글,목차,분야,정가,유사도,유사도 기준
0,상식 밖의 경제학(출간 10주년 기념판),댄 애리얼리,기존의 경제학을 대체할 새로운 경제학!\n행동경제학계의 대표주자인 댄 애리얼리 교수...,한국어판 출간 10주년 기념 서문\n\n1장 사람들은 비교를 좋아해\n- 왜 모든 ...,경제/경영,17000,0,소개글 기반 유사도
1,자본주의,EBS 자본주의 제작팀,우리가 반드시 알아야 할 자본주의의 진실!\n『자본주의』는 자본주의를 쉽게 풀어낸 ...,"프롤로그 _ 길 잃은 자본주의, 우리는 무엇을 할 수 있는가\n\n1장 ‘빚’이 있...",경제/경영,17000,0,소개글 기반 유사도
2,인간 실격(미니북)(초판본)(1948년 오리지널 초판본 표지디자인),다자이 오사무,20세기 일본 문학을 강타한 다자이 오사무의 대표작\n외면에 감춰진 젊은이의 비극적...,서문\n첫 번째 수기\n두 번째 수기\n세 번째 수기\n후기\n\n작품 해설 | 절...,소설,3900,0,소개글 기반 유사도



목차 유사도 기반


,상품명,저자,소개글,목차,분야,정가,유사도,유사도 기준
0,생각하지 않는 사람들(10주년),니콜라스 카,"“스마트 시대, 우리는 더 똑똑해지고 있는가?”\n\n세계적 경영컨설턴트 니콜라스 ...",개정판 서문\n서문: 감시견과 도둑\n\n1부 문자 혁명과 인간 사고의 확장\n\n...,경제/경영,20000,0,목차 기반 유사도
1,좌파의 길,낸시 프레이저,동시대 가장 독창적인 사회철학자\n낸시 프레이저의 뜨거운 제안-\n암울한 우리 시대...,감사의 글\n\n서문 _ ‘식인’이라는 은유\n\n1장 걸신들린 짐승: ‘자본주의’...,정치/사회,19500,0,목차 기반 유사도
2,21세기를 살아가는 반자본주의자를 위한 안내서,에릭 올린 라이트,현실적 유토피아를 찾아가는 지성의 낙관주의!\n\n21세기에 자본주의 잠식하기라는 ...,서문\n1장 왜 반자본주의자인가\n자본주의란 무엇인가\n자본주의에 반대하는 근거\n...,정치/사회,14000,0,목차 기반 유사도



제목 유사도 기반


,상품명,저자,소개글,목차,분야,정가,유사도,유사도 기준
0,차가운 자본주의,윤루카스,차갑지만 아름다운 자본주의 시장경제\n\n반년 만에 40만 경제 유튜버가 된\n윤루...,작가의 말_ 초판 절판시켜 버렸다\n프롤로그_ 당신을 위해 글을 쓰지 않는다\n\n...,경제/경영,17500,0,제목 기반 유사도
1,자본주의,EBS 자본주의 제작팀,우리가 반드시 알아야 할 자본주의의 진실!\n『자본주의』는 자본주의를 쉽게 풀어낸 ...,"프롤로그 _ 길 잃은 자본주의, 우리는 무엇을 할 수 있는가\n\n1장 ‘빚’이 있...",경제/경영,17000,0,제목 기반 유사도
2,자본주의와 장애,마타 러셀,“더 나은 미래를 구축할 수 있다는 희망을 위해서라도 러셀을 읽어야 한다!”\n_애...,발간사\n추천의 글\n편집자 서문\n\n들어가는 글\n자본주의와 장애인 권리운동\n...,정치/사회,17000,0,제목 기반 유사도


In [ ]:
intro_df = pd.DataFrame()
index_df = pd.DataFrame()
title_df = pd.DataFrame()

for title, t, result_df in zip(['소개글 기반 유사도', '목차 기반 유사도', '제목 기반 유사도'],
                                [intro_sim_list, index_sim_list, title_sim_list],
                                [intro_df, index_df, title_df]):
    for key, score in zip(t.index, t):
        if score == 0:
            continue
        book_info = Books.loc[key, ['상품명', '저자', '소개글', '목차', '분야', '정가']]
        book_info['유사도'] = score
        book_info['유사도 기준'] = title

        result_df = pd.concat([result_df, pd.DataFrame([book_info])], ignore_index=True)

상품명                                              방구석 미술관
저자                                                   조원재
소개글    세상에서 가장 술술 읽히는 미술책!\n《방구석 미술관》은 2018년 출간 이래 방송...
목차     들어가며\n01. 죽음 앞에 절규한 에드바르트 뭉크, 사실은 평균 수명을 높인 장수...
분야                                               예술/대중문화
정가                                                 16800
Name: 2, dtype: object

In [ ]:
text = """이 과제는 인류학적 접근을 통해 자본주의 경제 시스템이 인간 사회와 문화에 미치는 다양하고 복잡한
 영향을 체계적으로 탐구하며, 학생들은 자본주의의 형성과 확산, 소비문화, 불평등, 글로벌화, 환경파괴 등 다양한
  측면에서 인류학의 독특한 관점을 활용하여 현대 사회의 복잡한 사회 경제적 문제와 그 영향을 사회학적,
  문화인류학적, 공동체적 시각에서 심층적으로 이해하고 분석하며, 인간의 역사와 문화와 함께 성장한 자본주의와의
  관계에 대한 통찰력을 개발하고자 하는 목적을 달성하기 위한 레포트를 작성하도록 합니다."""